In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.stats import mode,zscore
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

data = pd.read_json("../../../../data/data.json")

data= data[data["capture"]== "tw2"]

data.sample(2)

,index,mac,timestamp,rssi,frequency,iq,angle,distance,capture,switch_order,ref,phases,aoa
33389,451,c300000a7a48,2024-11-29 17:35:42.932000+00:00,-50,2426,"[42, -200, 186, 61, -86, 170, -154, -129, 140,...",20,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[-1.3638041325999999, 0.3169040657, 2.03913873...","[1.698305885, -0.0406280377, -1.5267570231, -3...","[22.8944182458, 28.6454216913, 27.4600288222, ..."
30202,171,c300000a7a48,2024-11-29 14:21:49.011000+00:00,-55,2480,"[-149, 76, -39, -176, 171, -27, 2, 174, -167, ...",-9,300.0,tw2,2-2-2-0-5-6-14-13-8-10,"[2.6699238262, -1.788863588, -0.156601877, 1.5...","[-0.5048613717, -2.9537509432, -0.2598143698, ...","[7.7755701738, -15.7997442601, -22.1875235753,..."


# Coluna

Esse modelo usa como entrada cada coluna IQ que sabemos que existe no pacote, logo precisamos gerá-las

In [2]:
#2-2-2-0-5-6-14-13-8-10 = 8*2 elementos em cada coluna, 2*8 de ref
# CTE = 20*8 => 160-4-8= 148 => 148*2 = 74 =>74//8 => 9 com resto 2, logo precisamos ignorar a última amostra 
data["training"] = data["phases"].map(lambda x: np.array([np.array(x[i:i+4])[1:]-np.array(x[i:i+4])[:-1] for i in range(0,len(x),4)])[:-1]) 
print(len(data))
#data2 = data.copy()

C:\Users\pedro\AppData\Local\Temp\ipykernel_13408\667383325.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data["training"] = data["phases"].map(lambda x: np.array([np.array(x[i:i+4])[1:]-np.array(x[i:i+4])[:-1] for i in range(0,len(x),4)])[:-1])


23555


In [3]:
tsamples =[]
tlabels =[]

vsamples=[]
vlabels=[]

# Separamos no dataset para que a gnt seja capaz de depois analizar o conteúdo por pacote
tdataset_b, vdataset_b = train_test_split(data,test_size=0.3,random_state=10)

data2 = tdataset_b.copy()
data2["training"]*=-1
data2["angle"]*=-1
data2["aoa"]= data2["aoa"].map(lambda x: -1*np.array(x))

data= pd.concat((tdataset_b,data2))

tdataset, vdataset = train_test_split(data,test_size=0.3,random_state=10)

for row in tdataset.iterrows():
    for element in row[1]["training"]:
        tsamples.append(element)
        tlabels.append(row[1]["angle"])

for row in vdataset.iterrows():
    for element in row[1]["training"]:
        vsamples.append(element)
        vlabels.append(row[1]["angle"])

X_train = tsamples
X_val = vsamples


y_train = tlabels
y_val = vlabels 


In [4]:
len(X_train)

415494

In [5]:
tv=pd.DataFrame([{"angulo":angle, "treinamento":y_train.count(angle),"validacao" :y_val.count(angle)} for angle in set(y_val)])
tv.set_index("angulo")

,treinamento,validacao
angulo,,
0,26208,11700
1,5202,2034
2,5148,2448
3,5454,2448
4,5904,2088
5,5562,2070
6,5400,2268
7,5076,2304
8,4914,2628


In [6]:
model = xgb.XGBRegressor(enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
#Making predictions on the test set
predictions = model.predict(X_val)

#Calculating accuracy
mae_model =mean_absolute_error(y_val, predictions)
mse_model =mean_squared_error(y_val, predictions)

print(f"MAE= Modelo: {mae_model}")
print(f"MSE= Modelo: {mse_model}")

MAE= Modelo: 11.142810161820462
MSE= Modelo: 368.2624524691731


In [8]:
aoa_val=[]
results_labels= []
results_values = []
for row in vdataset.iterrows():
    pred=[]
    for element in row[1]["training"]:
        pred.append(model.predict([element])[0])
    results_labels.append((row[1]["angle"]))
    results_values.append(np.mean(pred))
    aoa_val.append(np.mean(row[1]["aoa"]))

In [9]:
#Calculating accuracy

mae_model =mean_absolute_error(results_labels, results_values)
mse_model =mean_squared_error(results_labels, results_values)

mae_aoa =mean_absolute_error(results_labels, aoa_val)
mse_aoa =mean_squared_error(results_labels, aoa_val)

print(f"MAE= Modelo: {mae_model}\t AoA={mae_aoa}")
print(f"MSE= Modelo: {mse_model}\t AoA={mse_aoa}")

MAE= Modelo: 9.595515448330621	 AoA=17.853757412268344
MSE= Modelo: 223.0124447783545	 AoA=801.0173406351859


In [10]:
#Calculating accuracy

aoa_val=[]
results_labels= []
results_values = []
for row in vdataset_b.iterrows():
    pred=[]
    for element in row[1]["training"]:
        pred.append(model.predict([element])[0])
    results_labels.append((row[1]["angle"]))
    results_values.append(np.mean(pred))
    aoa_val.append(np.mean(row[1]["aoa"]))

mae_model =mean_absolute_error(results_labels, results_values)
mse_model =mean_squared_error(results_labels, results_values)

mae_aoa =mean_absolute_error(results_labels, aoa_val)
mse_aoa =mean_squared_error(results_labels, aoa_val)

print(f"MAE= Modelo: {mae_model}\t AoA={mae_aoa}")
print(f"MSE= Modelo: {mse_model}\t AoA={mse_aoa}")

MAE= Modelo: 9.508590875059863	 AoA=17.91555958745627
MSE= Modelo: 221.0882798983188	 AoA=789.3761852881698


In [13]:
model.save_model("./pd_line_doubled.json")